In [ ]:
"""
how does distribution of hyp values differ by symbol?
"""
%matplotlib inline

import time
import json
import shutil
from collections import defaultdict

import h5py
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics.cluster

import torch

def run(filename):
    shutil.copyfile(f'../{filename}', '/tmp/foo.h5')
    f = h5py.File('/tmp/foo.h5', 'r')

    hypotheses_train = torch.from_numpy(f['hypotheses_train'][:])
    hypotheses_gnd_train = torch.from_numpy(f['gnd_hypotheses_train'][:].astype(np.uint8)).long()
    dsrefs_train = torch.from_numpy(f['dsrefs_train'][:].astype(np.uint8)).long()
    resdicts = f['resdicts']
    
    meta = json.loads(f['meta'][0])
    ref = meta['ref']
#     print(ref)

    U_gnd = hypotheses_gnd_train.size(0)
    U_pred = hypotheses_train.size(0)

    V = hypotheses_train.size(2)
#     print('U_gnd', U_gnd, 'V', V)
    
    N = dsrefs_train.size(0)

    render_start_id = 0
    render_end_id_excl = len(resdicts)

    batch_size = 128
#     print('_N', batch_size)

#     print('hypotheses_train.size()', hypotheses_train.size())

#     for n_start in [0, N - batch_size]:
#     for render_id in [render_start_id, render_end_id_excl - 1]:
    for render_id in [render_end_id_excl - 1]:
        n_start = render_id * batch_size
#         print('n_start', n_start)
        _hypotheses_train = hypotheses_train[:, n_start:n_start + batch_size]
#         print('_hypotheses_train.size()', _hypotheses_train.size())
    #     _hypotheses_gnd_train = hypotheses_gnd_train[:, -4:].contiguous()

        _N = _hypotheses_train.size(1)
#         print('_N', _N)

    #     values_by_u_by_m = defaultdict(dict)
    #     total_uv_samples = _N
    #     values_by_uv = torch.zeros(N, U_pred, V)

    #     _zeros = np.zeros
#         i = 1
#         plt.figure(figsize=(20, 10))
        plt.figure(figsize=(6, 4))
        for i, u in enumerate(range(U_pred)):
            for v in range(V):
                plt.scatter(np.ones(_N) * i, _hypotheses_train[u, :, v].numpy(), s=2)
    #             plt.title(f'u={u} v={v}')
                i += 0.03
#             i += 1
#         render_id = n_start // batch_size
        resdict = json.loads(resdicts[render_id])
        episode = resdict['episode']
        plt.title(f'{ref} episode={episode}')
        plt.show()

    f.close()

filenames = """
../hists/hypprop_with_predictor_eec54_hughtok1_20180929_094956.h5
../hists/hypprop_with_predictor_eec55_hughtok2_20180929_095009.h5
../hists/hypprop_with_predictor_eec56_hughtok33_20180929_095031.h5
../hists/hypprop_with_predictor_eec57_hughtok4_20180929_095455.h5
"""

filenames = filenames.split('\n')
filenames = [f.replace('../', '') for f in filenames if f != '']

for filename in filenames:
    run(filename)